In [1]:
import pandas as pd
import numpy as np

#### First we import and cleanup data. There is the costs database, countries, GDPs and regions that need to be joined.

In [74]:
#read cleaned up database
df = pd.read_excel(r"C:\Users\elife\Documents\NYU\CCP\ANALYSIS\240226_update_charts\240226_merged_costs.xlsx", sheet_name="Sheet1")
#read population, region and gdp data
ct = pd.read_csv(r"C:\Users\elife\Documents\NYU\CCP\countries_continents.csv")
gdp=pd.read_csv(r"C:\Users\elife\Documents\NYU\CCP\world bank statistics\gdp2020.csv")

In [75]:
#take fields that we will use
df=df[['Country','City','Phase','Line','Start year','End year','Length','TunnelPer','Tunnel','Stations','Real cost (2021 dollars)','Cost/km (2021 dollars)']]

In [76]:
#take columns we need, rename column to join with df
ct=ct[['ISO-3166', 'Country','Population','Region']]
ct = ct.rename({'ISO-3166': 'CO'}, axis='columns')
ct.head()

,CO,Country,Population,Region
0,AO,Angola,"30,809,762",Africa
1,BF,Burkina Faso,"19,751,535",Africa
2,BI,Burundi,"11,175,378",Africa
3,BJ,Benin,"11,485,048",Africa
4,BW,Botswana,"2,254,126",Africa


In [77]:
df['Country'].nunique() #number of countries in database

59

In [78]:
gdp = gdp.rename({'Country Name': 'Country','2020':'GDP in $'}, axis='columns')
gdp['GDP in $']=gdp['GDP in $'].round(0)
gdp['GDP in $']=gdp.apply(lambda x: "{:,}".format(x['GDP in $']), axis=1)

In [79]:
#take only lines with a country column value and count
df=df.dropna(how='all', subset=['Country'])
len(df)

944

In [80]:
#remove all rows with no or 0.0 cost value
df=df.dropna(axis=0,how='all', subset=['Cost/km (2021 dollars)'])
df=df.loc[~((df['Cost/km (2021 dollars)'] == 0))]

In [189]:
median_cost=df['Cost/km (2021 dollars)'].median()
median_cost

188.31137344515625

In [190]:
mean_cost=df['Cost/km (2021 dollars)'].mean()
mean_cost

243.5552483380065

In [194]:
#30% of cost data
df['Cost/km (2021 dollars)'].quantile(0.05)

81.3450389282798

In [81]:
len(df)

883

In [82]:
df = df.rename({'Country': 'CO'}, axis='columns')

In [83]:
len(df)

883

In [84]:
#combine all data
df=pd.merge(df, ct)

In [85]:
df['Country'].nunique()

59

In [86]:
df=pd.merge(df, gdp) 

In [87]:
df['Country'].nunique()

59

In [88]:
df['City'].nunique()

187

In [89]:
#replace "4 years","5 years", "?", "not started" values in column "Start year" with Nan, use a dictionary
df['Start year']=df['Start year'].replace(['4 years','5 years','?','not started',"X"],np.nan)

In [90]:
#replace "X" with Nan
df['End year']=df['End year'].replace(['X'],np.nan)
df['End year']=df['End year'].replace(['by 2026'], 2026)

#### Here we have all data in one dataframe. Now we need to create one group where all phases of one project is a single entry.

In [91]:
#defining the aggregate functions for group by per project (we will group the different phases of projects into single projects)
agg_functions={'Country':'first', 'City':'first','Line':'first','Phase':'count','Start year':'min', 'End year':'max','Length':'sum','Tunnel':'sum','Stations':'sum','Real cost (2021 dollars)':'sum', 'Population':'first','Region':'first','GDP in $':'first'}

In [92]:
#We are grouping the different phases of projects into single projects)
df_gr=df.groupby(['City','Line'], as_index=False).aggregate(agg_functions)
df_gr = df_gr.rename({'Phase':'Phases'}, axis='columns')
len(df_gr) #this gives us how many groups are created

684

In [93]:
df_gr[df_gr['City']=='Istanbul']

,Country,City,Line,Phases,Start year,End year,Length,Tunnel,Stations,Real cost (2021 dollars),Population,Region,GDP in $
251,Turkey,Istanbul,M1,2,1987.0,2027.0,36.30,19.00,33,5769.026586,"82,319,724",Middle East,"27,235.0"
252,Turkey,Istanbul,M11,2,2016.0,2022.0,69.50,69.50,16,6757.861508,"82,319,724",Middle East,"27,235.0"
253,Turkey,Istanbul,M12,1,2017.0,2023.0,13.00,13.00,11,2411.614259,"82,319,724",Middle East,"27,235.0"
254,Turkey,Istanbul,M2,1,1992.0,2014.0,23.50,21.50,17,7392.768478,"82,319,724",Middle East,"27,235.0"
255,Turkey,Istanbul,M3,3,2006.0,2023.0,31.20,30.50,23,4442.449719,"82,319,724",Middle East,"27,235.0"
256,Turkey,Istanbul,M4,3,2005.0,2022.0,33.60,33.60,23,3736.576528,"82,319,724",Middle East,"27,235.0"
257,Turkey,Istanbul,M4 P4-M10,1,2017.0,2024.0,14.00,14.00,8,1188.531956,"82,319,724",Middle East,"27,235.0"
258,Turkey,Istanbul,M5,2,2011.0,2027.0,37.50,37.50,29,4366.184309,"82,319,724",Middle East,"27,235.0"
259,Turkey,Istanbul,M6,1,2013.0,2015.0,3.30,3.30,4,285.236398,"82,319,724",Middle East,"27,235.0"
260,Turkey,Istanbul,M7,2,2014.0,2027.0,42.25,40.85,30,5945.749869,"82,319,724",Middle East,"27,235.0"


In [94]:
#Calculate Cost/km and tunnel percentage per project in the grouped section 
df_gr['Cost/km (2021 dollars)']=df_gr['Real cost (2021 dollars)']/df_gr['Length']
df_gr['TunnelPer']=df_gr['Tunnel']/df_gr['Length']
df_gr.sample(10)

,Country,City,Line,Phases,Start year,End year,Length,Tunnel,Stations,Real cost (2021 dollars),Population,Region,GDP in $,Cost/km (2021 dollars),TunnelPer
135,China,Dalian,Line 1,1,2010.0,2021.0,13.200,1.80,3,2040.344590,"1,392,730,000",East Asia,"17,189.0",154.571560,0.136364
140,Bangladesh,Dhaka,Line 1,1,2020.0,2026.0,31.200,19.90,19,16819.520000,"161,356,039",South Asia,"5,995.0",539.087179,0.637821
656,China,Xiamen,Line 2,2,2015.0,2020.0,40.700,40.70,28,8117.636357,"1,392,730,000",East Asia,"17,189.0",199.450525,1.000000
419,China,Qingdao,Line 1 + Line 7,1,NaN,2021.0,59.970,59.97,40,9557.610000,"1,392,730,000",East Asia,"17,189.0",159.373187,1.000000
287,Ukraine,Kyiv,M3,1,2018.0,2022.0,3.700,3.70,2,759.220854,"44,622,516",Europe,"13,087.0",205.194825,1.000000
46,China,Beijing,Yanfang Branch,1,2020.0,NaN,6.015,0.00,3,1126.027236,"1,392,730,000",East Asia,"17,189.0",187.203198,0.000000
653,China,Xi'an,Line 8,1,NaN,NaN,50.000,50.00,35,9657.850424,"1,392,730,000",East Asia,"17,189.0",193.157008,1.000000
374,China,Nanning,Line 5,1,NaN,2021.0,20.220,20.22,17,3919.600000,"1,392,730,000",East Asia,"17,189.0",193.847676,1.000000
589,Japan,Tokyo,Tama Monorail,1,1991.0,2000.0,16.000,0.00,19,2583.816299,"126,529,100",East Asia,"42,100.0",161.488519,0.000000
336,India,Mumbai,Line 3,1,2016.0,2022.0,33.500,33.50,27,15940.845743,"1,352,617,328",South Asia,"6,525.0",475.846142,1.000000


In [95]:
#Format the data
df_gr['Start year'] = df_gr['Start year'].astype( pd.Int64Dtype() )
df_gr['End year'] = df_gr['End year'].astype( pd.Int64Dtype() )
df_gr['Stations'] = df_gr['Stations'].astype( pd.Int64Dtype() )
df_gr['Real cost (2021 dollars)'] = df_gr['Real cost (2021 dollars)'].round(1)
df_gr["Cost/km (2021 dollars)"] = df_gr["Cost/km (2021 dollars)"].round(1)
df_gr["TunnelPer"] = df_gr["TunnelPer"].round(2)
df_gr['Tunnel Percentage']=(df_gr['TunnelPer']*100).astype(str)+" %"
df_gr['GDP in $'] = df_gr['GDP in $'].str.replace(",","").astype( float ).astype( pd.Int64Dtype() )
df_gr.head()

,Country,City,Line,Phases,Start year,End year,Length,Tunnel,Stations,Real cost (2021 dollars),Population,Region,GDP in $,Cost/km (2021 dollars),TunnelPer,Tunnel Percentage
0,Saudi Arabia,Ad Dammam,Dammam Metro,1,2014,<NA>,50.00,0.00,0,32269.5,"33,699,947",Middle East,46778,645.4,0.00,0.0 %
1,Turkey,Adana,First stage,1,1996,2010,13.90,3.50,13,1435.5,"82,319,724",Middle East,27235,103.3,0.25,25.0 %
2,India,Ahmadabad,Line,1,2015,2023,40.00,6.40,33,6326.5,"1,352,617,328",South Asia,6525,158.2,0.16,16.0 %
3,Netherlands,Amsterdam,Amsterdam North South,1,2003,2018,9.70,7.10,8,5090.1,"17,231,017",Europe,59267,524.7,0.73,73.0 %
4,Turkey,Ankara,M4,1,2003,2017,9.22,9.22,9,1366.9,"82,319,724",Middle East,27235,148.3,1.00,100.0 %


In [96]:
df_gr['Country'].nunique()

59

In [97]:
#create json dictionary. This is the format the vega library reads data in. Add data-elif to the top because that is what our map jsons have at the top.
data_dict = df_gr.to_dict(
    orient= 'records'
)

data_dict = {
    "data-elif": data_dict
}

data_dict

{'data-elif': [{'Country': 'Saudi Arabia',
   'City': 'Ad Dammam',
   'Line': 'Dammam Metro',
   'Phases': 1,
   'Start year': 2014,
   'End year': <NA>,
   'Length': 50.0,
   'Tunnel': 0.0,
   'Stations': 0,
   'Real cost (2021 dollars)': 32269.5,
   'Population': '33,699,947',
   'Region': 'Middle East',
   'GDP in $': 46778,
   'Cost/km (2021 dollars)': 645.4,
   'TunnelPer': 0.0,
   'Tunnel Percentage': '0.0 %'},
  {'Country': 'Turkey',
   'City': 'Adana',
   'Line': 'First stage',
   'Phases': 1,
   'Start year': 1996,
   'End year': 2010,
   'Length': 13.9,
   'Tunnel': 3.5,
   'Stations': 13,
   'Real cost (2021 dollars)': 1435.5,
   'Population': '82,319,724',
   'Region': 'Middle East',
   'GDP in $': 27235,
   'Cost/km (2021 dollars)': 103.3,
   'TunnelPer': 0.25,
   'Tunnel Percentage': '25.0 %'},
  {'Country': 'India',
   'City': 'Ahmadabad',
   'Line': 'Line',
   'Phases': 1,
   'Start year': 2015,
   'End year': 2023,
   'Length': 40.0,
   'Tunnel': 6.4,
   'Stations': 33

In [98]:
#save the data on file
import json
import os
folder = r'C:\Users\elife\Documents\NYU\CCP\ANALYSIS\240226_update_charts'
file = "data_phase_group.js"

# To export Np formats
def np_encoder(object):
    if isinstance(object, np.generic):
        return object.item()

with open(os.path.join(folder,file), "w") as outfile: 
    json_text = json.dumps(data_dict, indent=1 , default=np_encoder)
    test = "var data = " + json_text
    outfile.write( test )

print('done')

done


In [99]:
df.sample(10)

,CO,City,Phase,Line,Start year,End year,Length,TunnelPer,Tunnel,Stations,Real cost (2021 dollars),Cost/km (2021 dollars),Country,Population,Region,GDP in $
264,GR,Thessaloniki,Kalamaria Extension,Kalamaria Extension,2013.0,2021.0,4.8,1,4.8,5,814.833498,169.756979,Greece,"10,727,668",Europe,"27,949.0"
718,CN,Changchun,Line 2,Line 2,NaN,2020.0,22.8,1,22.8,19,3853.515430,169.013835,China,"1,392,730,000",East Asia,"17,189.0"
749,CN,Fuzhou,Line 6 East Extension,Line 6,2020.0,NaN,5.29,1,5.29,6,1021.682046,193.134602,China,"1,392,730,000",East Asia,"17,189.0"
43,CA,Vancouver,Expo Line Surrey-Langley extension,Expo Line,2022.0,NaN,16,0,0,8,3144.120000,196.507500,Canada,"37,058,856",N.America,"46,572.0"
791,CN,Nanjing,Line S4 Nanjing-Chuzhou Phase 2 (Chuzhou Section),Line S4 Nanjing-Chuzhou,2022.0,NaN,13.28,NaN,NaN,6,1056.380000,79.546687,China,"1,392,730,000",East Asia,"17,189.0"
807,CN,Shenzhen,Guangzhou-Dongguan-Shenzhen Regional Rail Sout...,Guangzhou-Dongguan-Shenzhen Regional Rail,2021.0,NaN,15.15,1,15.15,3,2844.100000,187.729373,China,"1,392,730,000",East Asia,"17,189.0"
55,US,New York,Second Avenue Phase 2,Second Avenue,2019.0,2029.0,2.6,1,2.6,3,6949.000000,2672.692308,United States,"327,167,434",N.America,"63,028.0"
35,CA,Toronto,Eglinton Crosstown,Eglinton Crosstown,2011.0,2022.0,19,0.53,10,25,4966.509435,261.395233,Canada,"37,058,856",N.America,"46,572.0"
90,FR,Paris,Line 12 to Aubervilliers,Line 12,2012.0,2021.0,1.7,1,1.7,2,294.189280,173.052517,France,"66,987,244",Europe,"46,991.0"
557,CN,Chongqing,Line 6 Branch (EXPO) Phase 2,Line 6,2016.0,2020.0,13.99,0.638313,8.93,7,1911.453914,136.630015,China,"1,392,730,000",East Asia,"17,189.0"


#### Now we go back to the ungrouped df and group by country

In [100]:
#defining the aggregate functions for group by per project (we will count all phases of projects separately)
agg2_functions={'Country':'first','Phase':'count','Length':'sum','Tunnel':'sum','Stations':'sum','Real cost (2021 dollars)':'sum', 'Population':'first','Region':'first','GDP in $':'first'}

In [101]:
df_gr_co=df.groupby(['Country'], as_index=False).aggregate(agg2_functions)
df_gr_co = df_gr_co.rename({'Phase': '# Records in Database'}, axis='columns')

In [102]:
df_gr_co.sort_values(by='# Records in Database', ascending=False)

,Country,# Records in Database,Length,Tunnel,Stations,Real cost (2021 dollars),Population,Region,GDP in $
10,China,495,12882.2566,9085.5316,7068,2.328258e+06,"1,392,730,000",East Asia,"17,189.0"
22,India,34,1184.1350,261.1910,1000,2.283231e+05,"1,352,617,328",South Asia,"6,525.0"
26,Italy,34,194.5000,174.4000,216,3.373683e+04,"60,431,283",Europe,"41,988.0"
52,Turkey,28,489.7700,354.0200,335,5.944759e+04,"82,319,724",Middle East,"27,235.0"
18,Germany,21,74.2000,65.2000,71,2.594427e+04,"82,927,922",Europe,"54,845.0"
56,United States,21,180.8000,62.0000,98,1.011273e+05,"327,167,434",N.America,"63,028.0"
46,South Korea,19,379.0510,260.4510,174,3.632598e+04,"51,635,256",East Asia,"45,226.0"
17,France,17,286.9000,231.2000,137,7.331591e+04,"66,987,244",Europe,"46,991.0"
50,Taiwan,17,381.0900,174.6000,312,1.404427e+05,"22,894,384",East Asia,"55,724.0"
8,Canada,16,191.3000,82.1000,157,5.069519e+04,"37,058,856",N.America,"46,572.0"


### Now we convert km to miles

In [103]:
df_gr_co['Lenght_in_miles']=(df_gr_co['Length']*0.621371).round(2)
df_gr_co['Tunnel_in_miles']=(df_gr_co['Tunnel']*0.621371).round(2)
df_gr_co["Avg Cost/miles"]=(df_gr_co['Real cost (2021 dollars)']/df_gr_co['Lenght_in_miles']).round(2)
df_gr_co["Avg Tunnel %"]= (df_gr_co['Tunnel']/df_gr_co['Length']).round(2)*100

In [104]:
df_gr_co["Avg Cost/km"]=(df_gr_co['Real cost (2021 dollars)']/df_gr_co['Length']).round(2)
#df_gr_co["Avg Tunnel %"]= (df_gr_co['Tunnel']/df_gr_co['Length']).round(2)*100

In [105]:
df_gr_co['GDP in $'] = df_gr_co['GDP in $'].str.replace(",","").astype( float ).astype( pd.Int64Dtype() )

In [106]:
df_gr_co['Length']=df_gr_co['Length'].astype( float ).round(2)
df_gr_co['Lenght_in_miles']=df_gr_co['Lenght_in_miles'].astype( float ).round(2)

In [107]:
df_gr_co['Avg Cost/km (all projects)']=(df_gr_co['Real cost (2021 dollars)'].sum()/df_gr_co['Length'].sum()).round(2)
df_gr_co['Avg Cost/miles (all projects)']=(df_gr_co['Real cost (2021 dollars)'].sum()/df_gr_co['Lenght_in_miles'].sum()).round(2)

In [108]:
#create json dictionary. This is the format the vega library reads data in. Add data-elif to the top because that is what our map jsons have at the top.
data_dict = df_gr_co.to_dict(
    orient= 'records'
)

data_dict = {
    "data-elif": data_dict
}

data_dict

{'data-elif': [{'Country': 'Argentina',
   '# Records in Database': 1,
   'Length': 20.0,
   'Tunnel': 20.0,
   'Stations': 4,
   'Real cost (2021 dollars)': 4646.0,
   'Population': '44,494,502',
   'Region': 'S.America',
   'GDP in $': 20769,
   'Lenght_in_miles': 12.43,
   'Tunnel_in_miles': 12.43,
   'Avg Cost/miles': 373.77,
   'Avg Tunnel %': 100.0,
   'Avg Cost/km': 232.3,
   'Avg Cost/km (all projects)': 211.85,
   'Avg Cost/miles (all projects)': 340.94},
  {'Country': 'Australia',
   '# Records in Database': 4,
   'Length': 83.5,
   'Tunnel': 47.9,
   'Stations': 28,
   'Real cost (2021 dollars)': 26839.480193769086,
   'Population': '24,992,369',
   'Region': 'Oceania',
   'GDP in $': 53317,
   'Lenght_in_miles': 51.88,
   'Tunnel_in_miles': 29.76,
   'Avg Cost/miles': 517.34,
   'Avg Tunnel %': 56.99999999999999,
   'Avg Cost/km': 321.43,
   'Avg Cost/km (all projects)': 211.85,
   'Avg Cost/miles (all projects)': 340.94},
  {'Country': 'Austria',
   '# Records in Database'

In [109]:
#save the data on file
import json
import os
folder = r'C:\Users\elife\Documents\NYU\CCP\ANALYSIS\240226_update_charts'
file = "data_country_group.js"

# To export Np formats
def np_encoder(object):
    if isinstance(object, np.generic):
        return object.item()

with open(os.path.join(folder,file), "w") as outfile: 
    json_text = json.dumps(data_dict, indent=1 , default=np_encoder)
    test = "var data = " + json_text
    outfile.write( test )

print('done')

done


### Calculate weighted average

In [110]:
total_length_miles=df_gr_co['Lenght_in_miles'].sum()
total_length_miles

12298.759999999998

In [111]:
total_length_km=df_gr_co['Length'].sum()
total_length_km

19792.900000000005

In [112]:
total_cost=df_gr_co['Real cost (2021 dollars)'].sum()
total_cost

4193164.468717305

In [113]:
WAv_CostpMiles=total_cost/total_length_miles
WAv_Costpkm=total_cost/total_length_km

In [114]:
print('Weighted Average of Cost/Km is '+WAv_Costpkm.round(2).astype(str))
print('Weighted Average of Cost/Miles is '+WAv_CostpMiles.round(2).astype(str))

Weighted Average of Cost/Km is 211.85
Weighted Average of Cost/Miles is 340.94


In [154]:
df['Duration']=df['End year']-df['Start year']
df['Duration/km']=df['Length']/df['Duration']
df.sample(10)

,CO,City,Phase,Line,Start year,End year,Length,TunnelPer,Tunnel,Stations,...,Cost/km (2021 dollars),Country,Population,Region,GDP in $,Duration,Duration/km,Detail,Detail Start,Detail End
526,CN,Xi'an,Line 15 Phase 1,Line 15,2020.0,2025.0,19,1,19,11,...,165.107801,China,"1,392,730,000",East Asia,"17,189.0",5.0,3.8,Xi'an Line 15 Phase 1,Xi'an Line 15 Phase 1 starts,Xi'an Line 15 Phase 1 ends
216,DK,Copenhagen,City Circle Line,City Circle Line,2009.0,2019.0,15.5,1,15.5,17,...,257.824875,Denmark,"5,797,446",Europe,"60,230.0",10.0,1.55,Copenhagen City Circle Line,Copenhagen City Circle Line starts,Copenhagen City Circle Line ends
497,CN,Fuzhou,Line 6,Line 6,2015.0,2019.0,40.5,0.790123,32,20,...,201.853809,China,"1,392,730,000",East Asia,"17,189.0",4.0,10.125,Fuzhou Line 6,Fuzhou Line 6 starts,Fuzhou Line 6 ends
641,CN,Wuxi,Line 1 South,Line 1,NaN,2019.0,5.187,1,5.187,3,...,126.346547,China,"1,392,730,000",East Asia,"17,189.0",NaN,NaN,Wuxi Line 1 South,Wuxi Line 1 South starts,Wuxi Line 1 South ends
521,CN,Hangzhou,Line 7,Line 7,2017.0,2021.0,45,NaN,NaN,22,...,182.604543,China,"1,392,730,000",East Asia,"17,189.0",4.0,11.25,Hangzhou Line 7,Hangzhou Line 7 starts,Hangzhou Line 7 ends
279,PH,Manila,Line 4,Line 4,2021.0,2025.0,15.6,0,0,11,...,209.070513,Philippines,"106,651,922",South East Asia,"8,394.0",4.0,3.9,Manila Line 4,Manila Line 4 starts,Manila Line 4 ends
239,IN,Mumbai,Line 12,Line 12,2020.0,2026.0,20.8,0,0,17,...,117.987324,India,"1,352,617,328",South Asia,"6,525.0",6.0,3.46667,Mumbai Line 12,Mumbai Line 12 starts,Mumbai Line 12 ends
16,TR,Istanbul,M5 Phase 2-M13,M5,2016.0,2027.0,17.8,1,17.8,13,...,117.362714,Turkey,"82,319,724",Middle East,"27,235.0",11.0,1.61818,Istanbul M5 Phase 2-M13,Istanbul M5 Phase 2-M13 starts,Istanbul M5 Phase 2-M13 ends
684,CN,Urumqi,Line 1,Line 1,2014.0,2019.0,27.615,1,27.615,21,...,207.555838,China,"1,392,730,000",East Asia,"17,189.0",5.0,5.523,Urumqi Line 1,Urumqi Line 1 starts,Urumqi Line 1 ends
804,CN,Shenyang,Line 9 Phase 1,Line 9,2012.0,2020.0,29.2,1,29.2,23,...,174.405597,China,"1,392,730,000",East Asia,"17,189.0",8.0,3.65,Shenyang Line 9 Phase 1,Shenyang Line 9 Phase 1 starts,Shenyang Line 9 Phase 1 ends


In [155]:
len(df)

883

In [170]:
#drop all rows with no Duration value
df_dur=df.dropna(axis=0,how='all', subset=['Duration'])


In [171]:
len(df_dur)

629

In [174]:
df_dur.sample(10)

,CO,City,Phase,Line,Start year,End year,Length,TunnelPer,Tunnel,Stations,...,Cost/km (2021 dollars),Country,Population,Region,GDP in $,Duration,Duration/km,Detail,Detail Start,Detail End
295,ES,Barcelona,Sabadell tunnel,Sabadell tunnel,2006.0,2017.0,4.4,1,4.4,4,...,153.043744,Spain,"46,723,749",Europe,"37,756.0",11.0,0.4,Barcelona Sabadell tunnel,Barcelona Sabadell tunnel starts,Barcelona Sabadell tunnel ends
30,CA,Toronto,Scarborough,Scarborough,2020.0,2030.0,7.8,1,7.8,3,...,639.160260,Canada,"37,058,856",N.America,"46,572.0",10.0,0.78,Toronto Scarborough,Toronto Scarborough starts,Toronto Scarborough ends
243,IN,Delhi,Phase 4 second half,Phase 4,2019.0,2025.0,42.3,0.35,14.7,43,...,239.292778,India,"1,352,617,328",South Asia,"6,525.0",6.0,7.05,Delhi Phase 4 second half,Delhi Phase 4 second half starts,Delhi Phase 4 second half ends
429,CN,Beijing,Line 7 Phase 2 (Eastern Extension),Line 7,2016.0,2020.0,17.2,0.872093,15,9,...,252.571726,China,"1,392,730,000",East Asia,"17,189.0",4.0,4.3,Beijing Line 7 Phase 2 (Eastern Extension),Beijing Line 7 Phase 2 (Eastern Extension) starts,Beijing Line 7 Phase 2 (Eastern Extension) ends
719,CN,Changchun,Line 1 South Extension,Line 1,2010.0,2019.0,8,1,8,6,...,203.521601,China,"1,392,730,000",East Asia,"17,189.0",9.0,0.888889,Changchun Line 1 South Extension,Changchun Line 1 South Extension starts,Changchun Line 1 South Extension ends
279,PH,Manila,Line 4,Line 4,2021.0,2025.0,15.6,0,0,11,...,209.070513,Philippines,"106,651,922",South East Asia,"8,394.0",4.0,3.9,Manila Line 4,Manila Line 4 starts,Manila Line 4 ends
332,JP,Tokyo,Sotetsu Shin-Yokohama Tokyu Line,Sotetsu Shin-Yokohama,2010.0,2022.0,10,0.8,8,2,...,311.899954,Japan,"126,529,100",East Asia,"42,100.0",12.0,0.833333,Tokyo Sotetsu Shin-Yokohama Tokyu Line,Tokyo Sotetsu Shin-Yokohama Tokyu Line starts,Tokyo Sotetsu Shin-Yokohama Tokyu Line ends
317,TH,Bangkok,Pink Line,Pink Line,2017.0,2022.0,34.5,0,0,30,...,141.157498,Thailand,"69,428,524",South East Asia,"18,198.0",5.0,6.9,Bangkok Pink Line,Bangkok Pink Line starts,Bangkok Pink Line ends
708,CN,Zhengzhou,Line 1 Phase 1,Line 1,2009.0,2013.0,25.2,1,25.2,20,...,194.619833,China,"1,392,730,000",East Asia,"17,189.0",4.0,6.3,Zhengzhou Line 1 Phase 1,Zhengzhou Line 1 Phase 1 starts,Zhengzhou Line 1 Phase 1 ends
730,CN,Chengdu,Line 3 Phase 2,Line 3,2013.0,2020.0,15,0.6,9,9,...,123.823886,China,"1,392,730,000",East Asia,"17,189.0",7.0,2.14286,Chengdu Line 3 Phase 2,Chengdu Line 3 Phase 2 starts,Chengdu Line 3 Phase 2 ends


#### We go back again and regroup by country, this time also taking into account the duration info. We didn't do this before not to lose entries without duration information.

In [175]:
#defining the aggregate functions for group by per project (we will group the different phases of projects into single projects)
agg3_functions={'Country':'first','Phase':'count','Length':'sum','Tunnel':'sum','Stations':'sum','Real cost (2021 dollars)':'sum', 'Population':'first','Region':'first','GDP in $':'first','Duration':'sum'}

In [183]:
df_gr_co_dur=df_dur.groupby(['Country'], as_index=False).aggregate(agg3_functions)
df_gr_co_dur = df_gr_co_dur.rename({'Phase': '# Records in Database'}, axis='columns')
df_gr_co_dur["Avg Cost/km"]=(df_gr_co_dur['Real cost (2021 dollars)']/df_gr_co['Length']).round(2)
df_gr_co_dur["Avg Tunnel %"]= ((df_gr_co_dur['Tunnel']/df_gr_co_dur['Length'])*100).round(2)
df_gr_co_dur['Length']=df_gr_co_dur['Length'].astype( float ).round(2)
df_gr_co_dur["Avg Duration/km"] = (df_gr_co_dur['Duration']/df_gr_co_dur['Length']).round(2)
df_gr_co_dur['GDP in $'] = df_gr_co_dur['GDP in $'].str.replace(",","").astype( float ).astype( pd.Int64Dtype() )
df_gr_co_dur['Avg Cost/km (all projects)']=(df_gr_co_dur['Real cost (2021 dollars)'].sum()/df_gr_co_dur['Length'].sum()).round(2)

In [184]:
len(df_gr_co_dur)

59

In [185]:
df_gr_co_dur['Real cost (2021 dollars)']= df_gr_co_dur['Real cost (2021 dollars)'].round(2)

In [186]:
df_gr_co_dur.sample(10)

,Country,# Records in Database,Length,Tunnel,Stations,Real cost (2021 dollars),Population,Region,GDP in $,Duration,Avg Cost/km,Avg Tunnel %,Avg Duration/km,Avg Cost/km (all projects)
28,Kuwait,1,160.0,56.000,68,31828.20,"4,137,309",Middle East,47303,5.0,198.93,35.00,0.03,230.16
43,Saudi Arabia,7,337.5,53.900,161,66817.12,"33,699,947",Middle East,46778,33.0,116.10,15.97,0.10,230.16
13,Dominican Republic,2,31.0,26.000,34,4926.51,"10,627,165",N.America,17934,12.0,158.92,83.87,0.39,230.16
20,Hong Kong,14,120.6,78.675,78,92038.22,"7,451,000",East Asia,58980,72.0,763.17,65.24,0.60,230.16
4,Bangladesh,3,72.1,33.900,49,41368.28,"161,356,039",South Asia,5995,21.0,573.76,47.02,0.29,230.16
12,Denmark,1,15.5,15.500,17,3996.29,"5,797,446",Europe,60230,10.0,257.82,100.00,0.65,230.16
45,Singapore,4,124.4,124.400,99,76796.07,"5,638,676",South East Asia,99681,37.0,617.33,100.00,0.30,230.16
37,Philippines,4,77.7,42.700,48,33657.30,"106,651,922",South East Asia,8394,22.0,433.17,54.95,0.28,230.16
17,France,17,286.9,231.200,137,73315.91,"66,987,244",Europe,46991,108.0,255.55,80.59,0.38,230.16
39,Portugal,2,8.0,4.800,9,681.20,"10,281,762",Europe,34177,6.0,85.15,60.00,0.75,230.16


In [187]:
#create json dictionary. This is the format the vega library reads data in. Add data-elif to the top because that is what our map jsons have at the top.
data_dict = df_gr_co_dur.to_dict(
    orient= 'records'
)

data_dict = {
    "data-elif": data_dict
}

data_dict

{'data-elif': [{'Country': 'Argentina',
   '# Records in Database': 1,
   'Length': 20.0,
   'Tunnel': 20.0,
   'Stations': 4,
   'Real cost (2021 dollars)': 4646.0,
   'Population': '44,494,502',
   'Region': 'S.America',
   'GDP in $': 20769,
   'Duration': 6.0,
   'Avg Cost/km': 232.3,
   'Avg Tunnel %': 100.0,
   'Avg Duration/km': 0.3,
   'Avg Cost/km (all projects)': 230.16},
  {'Country': 'Australia',
   '# Records in Database': 4,
   'Length': 83.5,
   'Tunnel': 47.9,
   'Stations': 28,
   'Real cost (2021 dollars)': 26839.48,
   'Population': '24,992,369',
   'Region': 'Oceania',
   'GDP in $': 53317,
   'Duration': 25.0,
   'Avg Cost/km': 321.43,
   'Avg Tunnel %': 57.37,
   'Avg Duration/km': 0.3,
   'Avg Cost/km (all projects)': 230.16},
  {'Country': 'Austria',
   '# Records in Database': 4,
   'Length': 30.7,
   'Tunnel': 24.5,
   'Stations': 19,
   'Real cost (2021 dollars)': 6497.32,
   'Population': '8,847,037',
   'Region': 'Europe',
   'GDP in $': 55686,
   'Duration

In [188]:
#save the data on file
import json
import os
folder = r'C:\Users\elife\Documents\NYU\CCP\ANALYSIS\240226_update_charts'
file = "data_country_dur_group.js"

# To export Np formats
def np_encoder(object):
    if isinstance(object, np.generic):
        return object.item()

with open(os.path.join(folder,file), "w") as outfile: 
    json_text = json.dumps(data_dict, indent=1 , default=np_encoder)
    test = "var data = " + json_text
    outfile.write( test )

print('done')

done


#### Timeline database

In [125]:
df['Detail']=df['City'].str.cat(df['Phase'],sep=" ")

In [133]:
df['Detail']

0                     Istanbul M4 Phase 3
1                             Istanbul M2
2                     Istanbul M3 Phase 1
3                     Istanbul M4 Phase 1
4                     Istanbul M4 Phase 2
                      ...                
878    Hong Kong Eastern Harbour Crossing
879                 Belgrade Metro Line 1
880                 Belgrade Metro Line 2
881                  Santo Domingo Line 1
882             Santo Domingo Line 2a, 2b
Name: Detail, Length: 883, dtype: object

In [134]:
df['Detail Start']=df['Detail'].astype(str)+" starts"

In [135]:
df['Detail End']=df['Detail'].astype(str)+" ends"

In [136]:
df01=df[['Country','Start year','Detail Start']]
df02=df[['Country','End year','Detail End']]

In [137]:
df01= df01.rename({'Start year': 'Year','Detail Start':'Detail'}, axis='columns')
df02= df02.rename({'End year': 'Year','Detail End':'Detail'}, axis='columns')

In [138]:
df_time = pd.concat([ df01,df02] )

In [139]:
df_time.sample(10)

,Country,Year,Detail
411,China,2019.0,Guangzhou Line 13 Phase 2 starts
217,Norway,2013.0,Oslo Løren Line starts
455,China,2017.0,Hangzhou Line 6 Phase 2 starts
216,Denmark,2019.0,Copenhagen City Circle Line ends
284,Iran,2014.0,Tehran Line 3 extension to SW ends
835,China,NaN,Xiamen Quanzhou-Xiamen-Zhangzhou Regional Rail...
190,South Korea,2019.0,Seoul Sin Ansan Line starts
170,Vietnam,2013.0,Hanoi Line 3 ends
188,South Korea,2018.0,Seoul GTX A starts
857,Egypt,1982.0,Cairo Line 1 starts


In [140]:
df_time.dropna(how='all', subset=['Year'])
df_time=df_time.groupby(['Country','Year'])['Detail'].apply(', '.join).reset_index()
df_time.sample(10)

,Country,Year,Detail
238,Israel,2022.0,Tel Aviv Red Line ends
197,Hong Kong,1998.0,Hong Kong Airport Railway ends
128,Finland,2015.0,Helsinki Ring Rail Line ends
134,France,2005.0,"Paris Line 13 to Courtilles starts, Lyon Line ..."
433,Taiwan,2026.0,Taipei Taoyuan MRT Green Line ends
482,United Kingdom,2009.0,London Crossrail starts
8,Australia,2024.0,Sydney Metro City & SW ends
91,China,2016.0,Beijing Capital Airport Express west extenion ...
258,Italy,2015.0,"Milan Line 4 starts, Catania Nesima-Monte Po s..."
283,Japan,2004.0,Tokyo Minatomirai Line ends


In [141]:
df_time['Year'] = df_time['Year'].astype( pd.Int64Dtype() )

In [142]:
df_time.sample(10)

,Country,Year,Detail
164,Germany,2009,Berlin U55 ends
465,Turkey,2019,Istanbul CR3 ends
171,Germany,2018,Hamburg U4 to Elbbrücken ends
286,Japan,2009,Osaka Hanshin Namba Line ends
467,Turkey,2023,"Istanbul M3 Phase 2 ends, Istanbul M12 ends, ..."
159,Germany,1997,Berlin U2 to Pankow starts
300,Malaysia,2017,Kuala Lumpur Kajang Line ends
99,China,2024,"Dongguan Line 3 feeder starts, Hefei Line 2 Ea..."
518,Vietnam,2029,Ho Chi Minh City Line 5 first phase ends
406,Sweden,2026,"Stockholm Barkarby ends, Gothenburg West Link ..."


In [143]:
#create json dictionary. This is the format the vega library reads data in. Add data-elif to the top because that is what our map jsons have at the top.
data_dict = df_time.to_dict(
    orient= 'records'
)

data_dict = {
    "data-elif": data_dict
}

data_dict

{'data-elif': [{'Country': 'Argentina',
   'Year': 2020,
   'Detail': 'Buenos Aires RER starts'},
  {'Country': 'Argentina', 'Year': 2026, 'Detail': 'Buenos Aires RER ends'},
  {'Country': 'Australia', 'Year': 2013, 'Detail': 'Sydney Metro NW starts'},
  {'Country': 'Australia',
   'Year': 2016,
   'Detail': 'Perth Forrestfield-Airport Link starts'},
  {'Country': 'Australia',
   'Year': 2017,
   'Detail': 'Sydney Metro City & SW starts'},
  {'Country': 'Australia',
   'Year': 2018,
   'Detail': 'Melbourne Metro Tunnel starts'},
  {'Country': 'Australia', 'Year': 2019, 'Detail': 'Sydney Metro NW ends'},
  {'Country': 'Australia',
   'Year': 2021,
   'Detail': 'Perth Forrestfield-Airport Link ends'},
  {'Country': 'Australia',
   'Year': 2024,
   'Detail': 'Sydney Metro City & SW ends'},
  {'Country': 'Australia',
   'Year': 2025,
   'Detail': 'Melbourne Metro Tunnel ends'},
  {'Country': 'Austria', 'Year': 1999, 'Detail': 'Vienna Lainz Tunnel starts'},
  {'Country': 'Austria',
   'Year

In [144]:
#save the data on file
import json
import os
folder = r'C:\Users\elife\Documents\NYU\CCP\ANALYSIS\240226_update_charts'
file = "data_time.js"

# To export Np formats
def np_encoder(object):
    if isinstance(object, np.generic):
        return object.item()

with open(os.path.join(folder,file), "w") as outfile: 
    json_text = json.dumps(data_dict, indent=1 , default=np_encoder)
    test = "var data = " + json_text
    outfile.write( test )

print('done')

done


In [145]:
agg5_functions={'Country':'first','Start year':'first','Length':'sum','Real cost (2021 dollars)':'sum','Phase':'count'}
df_heat=df.groupby(['Country','Start year'], as_index=False).aggregate(agg5_functions)
df_heat['Start year'] = df_heat['Start year'].astype( pd.Int64Dtype() )
df_heat.sample(10)

,Country,Start year,Length,Real cost (2021 dollars),Phase
51,China,2007,264.118,50360.239649,9
36,Canada,2022,16.000,3144.120000,1
214,Singapore,2002,35.500,12127.796587,1
278,Turkey,2018,32.000,3164.811836,1
147,Italy,1999,7.600,3143.689327,2
151,Italy,2003,15.800,1348.616251,2
260,Thailand,2020,22.100,4103.394013,1
59,China,2015,380.860,73522.354417,15
109,Germany,2022,3.800,1333.800000,1
4,Australia,2018,9.000,8683.236000,1


In [146]:
df_heat= df_heat.rename({'Phase': '# Projects starting'}, axis='columns')
df_heat['Mean Cost/km']=(df_heat['Real cost (2021 dollars)']/df_heat['Length']).round(2)

In [147]:
df_heat['Square Root of Cost/km']=df_heat['Mean Cost/km']**(1/2)
df_heat.sample(10)

,Country,Start year,Length,Real cost (2021 dollars),# Projects starting,Mean Cost/km,Square Root of Cost/km
78,Egypt,2006,4.30,3789.136606,1,881.19,29.684845
281,United Arab Emirates,2005,35.00,10489.161290,1,299.69,17.311557
247,Taiwan,2006,51.00,9106.627356,1,178.56,13.362634
52,China,2008,94.52,21953.816840,3,232.27,15.240407
236,Spain,2020,10.50,988.000000,1,94.10,9.700515
83,France,1991,2.50,590.906046,1,236.36,15.374004
289,United States,2007,7.10,21100.692518,3,2971.93,54.515411
267,Turkey,2003,9.22,1366.928679,1,148.26,12.176206
293,United States,2013,25.90,5768.186324,2,222.71,14.923471
103,Germany,2010,7.00,2919.164794,2,417.02,20.421068


In [148]:
#create json dictionary. This is the format the vega library reads data in. Add data-elif to the top because that is what our map jsons have at the top.
data_dict = df_heat.to_dict(
    orient= 'records'
)

data_dict = {
    "data-elif": data_dict
}

data_dict

{'data-elif': [{'Country': 'Argentina',
   'Start year': 2020,
   'Length': 20.0,
   'Real cost (2021 dollars)': 4646.0,
   '# Projects starting': 1,
   'Mean Cost/km': 232.3,
   'Square Root of Cost/km': 15.241391012633985},
  {'Country': 'Australia',
   'Start year': 2013,
   'Length': 36.0,
   'Real cost (2021 dollars)': 6076.325023853472,
   '# Projects starting': 1,
   'Mean Cost/km': 168.79,
   'Square Root of Cost/km': 12.991920566259632},
  {'Country': 'Australia',
   'Start year': 2016,
   'Length': 8.5,
   'Real cost (2021 dollars)': 1384.9191699156136,
   '# Projects starting': 1,
   'Mean Cost/km': 162.93,
   'Square Root of Cost/km': 12.764403628842203},
  {'Country': 'Australia',
   'Start year': 2017,
   'Length': 30.0,
   'Real cost (2021 dollars)': 10695.0,
   '# Projects starting': 1,
   'Mean Cost/km': 356.5,
   'Square Root of Cost/km': 18.881207588499205},
  {'Country': 'Australia',
   'Start year': 2018,
   'Length': 9.0,
   'Real cost (2021 dollars)': 8683.235999

In [149]:
#save the data on file
import json
import os
folder = r'C:\Users\elife\Documents\NYU\CCP\ANALYSIS\240226_update_charts'
file = "data_heat.js"

# To export Np formats
def np_encoder(object):
    if isinstance(object, np.generic):
        return object.item()

with open(os.path.join(folder,file), "w") as outfile: 
    json_text = json.dumps(data_dict, indent=1 , default=np_encoder)
    test = "var data = " + json_text
    outfile.write( test )

print('done')

done


In [150]:
len(df_gr_co)

59

In [151]:
df_gr_co['GDP in $'].median()

34287.0

In [152]:
df_gr_co[['Country','GDP in $']].sort_values(by=['GDP in $'])

,Country,GDP in $
34,Pakistan,5426
4,Bangladesh,5995
22,India,6525
57,Uzbekistan,7746
37,Philippines,8394
14,Ecuador,10890
58,Vietnam,10904
36,Peru,11908
23,Indonesia,12072
15,Egypt,12606
